## ==================== Первая часть решения =======================

In [3]:
!pip install -q sentence-transformers datasets scikit-learn lightgbm joblib transformers nlpaug evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00


In [1]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 17.5 MB/s eta 0:00:00


In [2]:
print('Collab is working')

Collab is working


In [4]:
import pandas as pd
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from tqdm import tqdm
import re
import time
import json
from sklearn.metrics import accuracy_score, f1_score
from sentence_transformers import SentenceTransformer
import numpy as np
import time
from sklearn.model_selection import train_test_split
from datasets import Dataset
from nltk.stem.snowball import SnowballStemmer
import pandas as pd
import random
import re
from tqdm import tqdm
from transformers import MarianMTModel, MarianTokenizer, AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig
import evaluate
from transformers import EarlyStoppingCallback
from torch.utils.data import DataLoader
import transformers, peft, accelerate

In [5]:
# Settings

USE_AUG = True
USE_KEYWORDS = True
USE_BNB = False

In [6]:
# Some initialisation

tqdm.pandas()
device = 0 if torch.cuda.is_available() else -1
print("CUDA:", torch.cuda.is_available())

CUDA: True


### Чиатем файлы

In [7]:
df = pd.read_csv("train.csv")
df = df.dropna(subset=["text"]).reset_index(drop=True)
df["text"] = df["text"].map(lambda t: str(t).strip())

In [8]:
categories = []
with open('categories.txt', 'r') as f:
    categories = f.read().split('\n')
categories

['бытовая техника',
 'обувь',
 'одежда',
 'посуда',
 'текстиль',
 'товары для детей',
 'украшения и аксессуары',
 'электроника',
 'нет товара']

### Напишем функционал для того, чтобы относить отзыв к категории по явному включению слова, если оно есть

In [9]:
stemmer = SnowballStemmer("russian")

keywords_raw = {
    "бытовая техника": [
        "пылесос", "чайник", "миксер", "блендер", "холодильник", "морозильник"
    ],
    "электроника": [
        "телефон", "смартфон", "планшет", "ноутбук", "компьютер", "монитор",
        "телевизор", "камера", "наушник", "колонк", "смарт-час", "гаджет"
    ]
}

keywords = {cls: [stemmer.stem(k) for k in keys] for cls, keys in keywords_raw.items()}

def keyword_match(text):
    text_low = text.lower()
    tokens = re.findall(r"[а-яё]+", text_low)
    stems = [stemmer.stem(t) for t in tokens]

    matched = []
    for cls, stems_list in keywords.items():
        if any(k in stems for k in stems_list):
            matched.append(cls)
    return matched

### Разметка текста ансамблем

In [10]:
model_names = [
    "joeddav/xlm-roberta-large-xnli",
    "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli",
    "MoritzLaurer/deberta-v3-large-zeroshot-v2.0"
]

classifiers = [pipeline("zero-shot-classification", model=mid, device=device) for mid in model_names]

tokenizer_zsl  = AutoTokenizer.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
model_zsl      = AutoModelForSequenceClassification.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
zsl = pipeline("zero-shot-classification", model=model_zsl, tokenizer=tokenizer_zsl, device=device)

def ensemble_classify(text, classifiers, categories, weights=[0.6,0.2,0.2]):
    probs = np.zeros(len(categories))
    for clf, w in zip(classifiers, weights):
        res = clf(text, candidate_labels=categories, multi_label=False)
        scores = np.array([res["scores"][res["labels"].index(cat)] for cat in categories])
        probs += w * scores
    probs /= probs.sum()
    top_idx = np.argmax(probs)
    return categories[top_idx], float(probs[top_idx])

def classify_text(text, classifiers, categories, threshold=0.5):
    if (USE_KEYWORDS):
        kws = keyword_match(text)
        if len(kws) == 1:
            return kws[0], 1.0, "keyword"
        else:
            label, conf = ensemble_classify(text, classifiers, categories)
            if label != "нет товара" and conf < threshold:
                label = "нет товара"
            return label, conf, "ensemble"
    else:
        label, conf = ensemble_classify(text, classifiers, categories)
        if label != "нет товара" and conf < threshold:
            label = "нет товара"
        return label, conf, "ensemble"

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

Device set to use cuda:0
Device set to use cuda:0


In [11]:
BATCH_SIZE = 32
labels, confs, methods = [], [], []

for i in tqdm(range(0, len(df), BATCH_SIZE)):
    batch_texts = df["text"].iloc[i:i+BATCH_SIZE].tolist()

    for text in batch_texts:
        label, conf, method = classify_text(text, classifiers, categories, threshold=0.5)
        labels.append(label)
        confs.append(conf)
        methods.append(method)

df["label"] = labels
df["conf"] = confs
df["method"] = methods

100%|██████████| 57/57 [24:27<00:00, 25.75s/it]


In [12]:
print(df['label'].value_counts())

label
нет товара                1093
одежда                     499
текстиль                   179
обувь                       20
товары для детей            14
украшения и аксессуары       6
посуда                       5
электроника                  2
Name: count, dtype: int64


### Бедем рассматривать только уверенные предсказания

In [13]:
THRESH = 0.6
df = df[df["conf"] >= THRESH].reset_index(drop=True)
print(df['label'].value_counts())

label
одежда                    351
нет товара                263
текстиль                  135
обувь                      11
товары для детей            8
посуда                      4
украшения и аксессуары      3
электроника                 2
Name: count, dtype: int64


### Сделаем аугментацию, если включена настройка

In [14]:
if (USE_AUG):
    AUG_TARGET_PER_CLS = 12
    AUG_CAP_PER_CLS    = 12
    AUG_PER_SAMPLE     = 2

    bt_model_ru_en = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
    bt_tok_ru_en   = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

    bt_model_en_ru = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
    bt_tok_en_ru   = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")

    def backtranslate(text: str) -> str:
        try:
            en = bt_model_ru_en.generate(**bt_tok_ru_en(text, return_tensors="pt", truncation=True))
            en_text = bt_tok_ru_en.decode(en[0], skip_special_tokens=True)

            ru = bt_model_en_ru.generate(**bt_tok_en_ru(en_text, return_tensors="pt", truncation=True))
            ru_text = bt_tok_en_ru.decode(ru[0], skip_special_tokens=True)
            return ru_text
        except Exception:
            return text

    para_model = AutoModelForSeq2SeqLM.from_pretrained("cointegrated/rut5-base-paraphraser")
    para_tok   = AutoTokenizer.from_pretrained("cointegrated/rut5-base-paraphraser")


    def paraphrase(text: str) -> str:
        try:
            inputs = para_tok(f"paraphrase: {text}", return_tensors="pt", truncation=True, max_length=128)
            ids = para_model.generate(**inputs, num_beams=4, max_length=128, num_return_sequences=1)
            return para_tok.decode(ids[0], skip_special_tokens=True)
        except Exception:
            return text

    def is_clean(text: str) -> bool:
        if len(text.split()) < 3:
            return False
        if re.search(r"[^а-яА-Яa-zA-Z0-9 ,.!?()-]", text):
            return False
        return True

    def augment_text(text: str, method: str) -> str:
        if method == "bt":
            return backtranslate(text)
        elif method == "para":
            return paraphrase(text)
        else:
            return text

    augmented_parts = []
    aug_counter = {}

    for cls in categories:
        subset = df[df["label"] == cls].copy()
        n = len(subset)

        if n == 0:
            aug_counter[cls] = 0
            continue

        need = max(0, min(AUG_TARGET_PER_CLS, AUG_CAP_PER_CLS) - n)
        if need == 0:
            aug_counter[cls] = 0
            continue

        gen_texts = []
        attempts = 0
        with tqdm(total=need) as pbar:
            while len(gen_texts) < need and attempts < need * 10:
                attempts += 1
                row = subset.sample(1).iloc[0]
                text = row["text"]

                method = random.choice(["bt", "para"])
                new_text = augment_text(text, method)

                if not is_clean(new_text) or new_text == text:
                    continue

                gen_texts.append({
                    "text": new_text,
                    "label": cls,
                    "conf": row["conf"] * 0.7,
                    "method": f"augment-{method}"
                })
                pbar.update(1)

        if gen_texts:
            aug_df = pd.DataFrame(gen_texts)
            augmented_parts.append(aug_df)
            aug_counter[cls] = len(aug_df)
        else:
            aug_counter[cls] = 0

    augmented = pd.concat([df] + augmented_parts, ignore_index=True)
    augmented = augmented.sample(frac=1, random_state=42).reset_index(drop=True)
    augmented.to_csv('aug.csv')

    print(pd.Series(aug_counter).to_string())
    print(augmented["label"].value_counts())

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/307M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
100%|██████████| 10/10 [00:42<00:00,  4.

бытовая техника            0
обувь                      1
одежда                     0
посуда                     8
текстиль                   0
товары для детей           4
украшения и аксессуары     9
электроника               10
нет товара                 0
label
одежда                    351
нет товара                263
текстиль                  135
товары для детей           12
украшения и аксессуары     12
электроника                12
обувь                      12
посуда                     12
Name: count, dtype: int64


In [15]:
if not USE_AUG:
    df.to_csv('no_aug.csv')

## ==================== Вторая часть решения =======================

### Читаем данные

In [16]:
train = pd.read_csv("aug.csv") if USE_AUG else pd.read_csv("no_aug.csv")
test = pd.read_csv("test.csv")

### Разделяем данные на train/validation так, чтобы классы распределились +- равномерно

In [17]:
TEXT_COL = "text"
LABEL_COL = "label"
VAL_SIZE = 0.2
MAX_VAL_PER_CLASS = 10

train[LABEL_COL] = train[LABEL_COL].astype(str)
labels = sorted(train[LABEL_COL].unique().tolist())
num_labels = len(labels)

label2id = {l:i for i,l in enumerate(labels)}
id2label = {i:l for l,i in label2id.items()}

train_parts, val_parts = [], []

for cls, grp in train.groupby(LABEL_COL):
    n = len(grp)
    val_n = max(1, int(n * VAL_SIZE))
    val_n = min(val_n, MAX_VAL_PER_CLASS)

    val_cls = grp.sample(n=val_n, random_state=42)
    train_cls = grp.drop(val_cls.index)

    val_parts.append(val_cls)
    train_parts.append(train_cls)

train_df = pd.concat(train_parts).sample(frac=1, random_state=42).reset_index(drop=True)
val_df = pd.concat(val_parts).sample(frac=1, random_state=42).reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
print(len(train_df), len(val_df))

print("Train df\n", train_df[LABEL_COL].value_counts())
print("Validation df\n", val_df[LABEL_COL].value_counts())


769 40
Train df
 label
одежда                    341
нет товара                253
текстиль                  125
электроника                10
посуда                     10
украшения и аксессуары     10
обувь                      10
товары для детей           10
Name: count, dtype: int64
Validation df
 label
одежда                    10
текстиль                  10
нет товара                10
украшения и аксессуары     2
электроника                2
товары для детей           2
обувь                      2
посуда                     2
Name: count, dtype: int64


### Токенизируем данные

In [18]:
def clean_text(t):
    if pd.isna(t):
        return ""
    return str(t).strip()

def to_dataset(df):
    ds = Dataset.from_pandas(df[[TEXT_COL, LABEL_COL]].rename(columns={TEXT_COL:"text", LABEL_COL:"label"}))
    ds = ds.map(lambda x: {"label": label2id[x["label"]]})
    return ds

In [19]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

train_df[TEXT_COL] = train_df[TEXT_COL].map(clean_text)
val_df[TEXT_COL]   = val_df[TEXT_COL].map(clean_text)
test[TEXT_COL]     = test[TEXT_COL].map(clean_text)

train_ds = to_dataset(train_df)
val_ds   = to_dataset(val_df)

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, max_length=256)

train_ds = train_ds.map(tokenize_fn, batched=True)
val_ds   = val_ds.map(tokenize_fn, batched=True)
train_ds.set_format(type="torch", columns=["input_ids","attention_mask","label"])
val_ds.set_format(type="torch", columns=["input_ids","attention_mask","label"])

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/769 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/769 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [21]:
if USE_BNB:
    quant_cfg = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4"
    )
else:
    quant_cfg = None

backbone = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path="xlm-roberta-base",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    quantization_config=quant_cfg
)

backbone.to(device)
backbone = prepare_model_for_kbit_training(backbone)

peft_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["query", "value"],
    bias="none",
    task_type=TaskType.SEQ_CLS
)

model = get_peft_model(backbone, peft_cfg)
model.to(device)

train_cfg = TrainingArguments(
    output_dir="./traindir",
    learning_rate=1.5e-4,
    warmup_ratio=0.1,
    weight_decay=0.05,
    per_device_eval_batch_size=64,
    per_device_train_batch_size=16,
    num_train_epochs=10,
    max_grad_norm=1.0,
    label_smoothing_factor=0.05,
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=100,
    eval_steps=100,
    logging_steps=20,
    fp16=torch.cuda.is_available(),
    load_best_model_at_end=True,
    optim="paged_adamw_8bit" if quant_cfg is not None else "adamw_torch",
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none",
    lr_scheduler_type="cosine",
)


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
f1_metric = evaluate.load("f1")

def eval_metrics(prediction_output):
    logits, gold_labels = prediction_output
    predicted = logits.argmax(axis=-1)
    result = f1_metric.compute(
        predictions=predicted,
        references=gold_labels,
        average="weighted"
    )
    return {"f1_weighted": result["f1"]}

for round_idx in range(2):
    trainer_obj = Trainer(
        model=model,
        args=train_cfg,
        tokenizer=tokenizer,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        compute_metrics=eval_metrics,
        callbacks=[
            EarlyStoppingCallback(
                early_stopping_threshold=5e-4,
                early_stopping_patience=2
            )
        ]
    )

    trainer_obj.train()


/tmp/ipython-input-3979272393.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_obj = Trainer(


Step,Training Loss,Validation Loss,F1 Weighted
100,0.954200,1.409401,0.560577
200,0.592800,1.221225,0.600198
300,0.564100,1.035888,0.665674
400,0.489600,0.995840,0.665089


/tmp/ipython-input-3979272393.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_obj = Trainer(


Step,Training Loss,Validation Loss,F1 Weighted
100,0.443300,0.787910,0.823344
200,0.370600,0.865524,0.803706
300,0.385300,0.740864,0.827165
400,0.329900,0.727649,0.827165


### Смотрим метрику F1 на валидации и калибруем категорию "нет товара"

In [28]:
def get_probabilities(batch_texts, batch_sz=32):
    encodings = tokenizer(batch_texts,
                    truncation=True,
                    padding=True,
                    max_length=256,
                    return_tensors="pt")

    tensor_ds = torch.utils.data.TensorDataset(encodings["input_ids"], encodings["attention_mask"])
    dl = DataLoader(tensor_ds, batch_size=batch_sz)

    collected = []
    model.eval()
    with torch.no_grad():
        for ids, mask in dl:
            if torch.cuda.is_available():
                ids, mask = ids.cuda(), mask.cuda()
            output = model(input_ids=ids, attention_mask=mask)
            probas = torch.softmax(output.logits, dim=-1).cpu().numpy()
            collected.append(probas)

    return np.vstack(collected)

texts_val = val_df['text'].tolist()
true_labels = [label2id[label] for label in val_df['label'].tolist()]

pred_probas = get_probabilities(texts_val, batch_sz=32)
pred_labels = pred_probas.argmax(axis=1)

print("F1:", f1_score(true_labels, pred_labels, average="weighted"))

idx_none_class = label2id.get("нет товара", None)
if idx_none_class is not None:
    candidate_probs = pred_probas.max(axis=1)
    opt_threshold, opt_f1 = 0.0, -1

    for threshold in np.linspace(0.1, 0.8, 15):
        adjusted_preds = pred_labels.copy()
        adjusted_preds[candidate_probs < threshold] = idx_none_class
        score = f1_score(true_labels, adjusted_preds, average="weighted")
        if score > opt_f1:
            opt_f1 = score
            opt_threshold = threshold

else:
    opt_threshold = None


F1: 0.8271645021645021


In [30]:
(true_labels == pred_labels).sum(), (true_labels != pred_labels).sum()

(np.int64(34), np.int64(6))

### Формируем выходной файл и считаем время, за которые это делается

In [35]:
test_texts = test['text'].tolist()
t0 = time.perf_counter()
probs_test = get_probabilities(test_texts, batch_sz=32)
t1 = time.perf_counter()
total = t1 - t0
avg_time = total / len(test_texts)
print(f"Общее время предсказания теста: {total}сек, среднее на 1 сэпмл: {avg_time}сек")
print('Таким образом, в критерии укладываемся')

pred_idx = probs_test.argmax(axis=1)
max_probs = probs_test.max(axis=1)
if opt_threshold:
    pred_idx[max_probs < opt_threshold] = idx_none_class

pred_labels = [id2label[i] for i in pred_idx]
out = pd.DataFrame({"text": test[TEXT_COL], "pred": pred_labels, "conf": max_probs})
out.to_csv("lora.csv", index=False)
print(out['pred'].value_counts())

Общее время предсказания теста: 15.30449929400038сек, среднее на 1 сэпмл: 0.002103422112974214сек
Таким образом, в критерии укладываемся
pred
одежда                    3406
нет товара                2172
текстиль                  1089
обувь                      275
посуда                     178
украшения и аксессуары      76
товары для детей            49
электроника                 31
Name: count, dtype: int64


In [ ]:
result = pd.DataFrame({"category": pred_labels})
result.to_csv('result.csv')

## ====================== Очистка памяти =========================


In [ ]:
import torch
import gc
import os

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

gc.collect()

from transformers import file_utils
cache_dir = file_utils.TRANSFORMERS_CACHE
if os.path.exists(cache_dir):
    !rm -rf {cache_dir}/*

In [ ]:
!rm -rf /root/.cache/huggingface/hub/*
!rm -rf /root/.cache/huggingface/transformers/*
!rm -rf /root/.cache/torch/*

import shutil
cache_dirs = [
    '/root/.cache/huggingface',
    '/root/.cache/torch',
    '/root/.cache/selenium'
]

for cache_dir in cache_dirs:
    if os.path.exists(cache_dir):
        shutil.rmtree(cache_dir)
        os.makedirs(cache_dir, exist_ok=True)

## ====================== Подвал =========================

In [ ]:
!pip uninstall -y bitsandbytes